In [5]:
## Creates .csv list of crypto tickers        ## As of 2021/04/30 There are: 4904 cryptos ##

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from tqdm.notebook import tqdm
import time

driver = webdriver.Chrome('C:/WebDriver/chromedriver')

driver.get("https://coinmarketcap.com/all/views/all/")

load_more = driver.find_element_by_xpath("//div/button[text()='Load More']")

#close popup
alert_x = driver.find_element_by_xpath("//*[@id='cmc-cookie-policy-banner']/div[2]")
alert_x.click()

    ## Begin Scrolling ##
start = time.perf_counter()
print("Phase[1/2]--Scrolling", end=' ')

while(True):
    try:   
        load_more.click()
    except:
        break
        
end = time.perf_counter()
p1_time = divmod(int(end-start), 60)
print("[{:02}:{:02}]".format(p1_time[0], p1_time[1]))

    ## Begin Scraping ##
rows = driver.find_elements_by_xpath("//tr[contains(@class, 'cmc-table-row')]")
driver.close()

with open("./../Tickers/tickers_crypto.csv", "w") as f:
    for i in tqdm(range(len(rows)), desc = 'Phase [2/2] -- Scraping'):
        
        try:
            symbol = rows[i].find_element_by_xpath("td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol']/div").text
            f.write(symbol + "\n")
        except:
            pass
            
            

Phase[1/2]--Scrolling [01:15]
